In [18]:
import numpy as np
import os

In [19]:
def write_pbs_head(fpbs, job_name):
    fpbs.write('#! /bin/bash\n')
    fpbs.write('#PBS -M zhangji@csrc.ac.cn\n')
    fpbs.write('#PBS -l nodes=1:ppn=24\n')
    fpbs.write('#PBS -l walltime=72:00:00\n')
    fpbs.write('#PBS -q common\n')
    fpbs.write('#PBS -N %s\n' % job_name)
    fpbs.write('\n')
    fpbs.write('cd $PBS_O_WORKDIR\n')
    fpbs.write('\n')
    

In [23]:
# case 1, traverse all parameters, inf space
maxtheta_list = [10, 50, 100, 500, 1000, 5000, 10000, 50000]
nphi_list = [100, 500, 1000, 5000, 10000, 20000]
nthfct_list = [1, 10, 100]  # nth = nthfct * maxtheta
job_dir = 'infhlx_20180510'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

t_name = os.path.join(job_dir, 'run.sh')
with open(t_name, 'w') as frun:
    # create .pbs file
    for maxtheta in maxtheta_list:
        for nphi in nphi_list:
            for nthfct in nthfct_list:
                nth = nthfct * maxtheta
                job_name = 'infhlx%06d_%06d_%04d' % (maxtheta, nphi, nthfct)
                t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../try_infhelix.py ')
                    fpbs.write(' -infhelix_maxtheta %f ' % maxtheta)
                    fpbs.write(' -infhelix_nnode %f ' % nphi)
                    fpbs.write(' -infhelix_ntheta %f ' % nth)
                    fpbs.write(' > %s.txt' % job_name)
                
    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
    
    frun.write('\n')

In [37]:
# case 2, maxtheta varing, inf space
maxtheta_list = 10 ** np.arange(1, 8)
nphi_list = [100, ]
nthfct_list = [30, ]  # nth = nthfct * maxtheta
job_dir = 'infhlx_20180511'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

t_name = os.path.join(job_dir, 'run.sh')
with open(t_name, 'w') as frun:
    # create .pbs file
    for maxtheta in maxtheta_list:
        for nphi in nphi_list:
            for nthfct in nthfct_list:
                nth = nthfct * maxtheta
                job_name = 'infhlx_maxtheta1E%02d' % np.log10(maxtheta)
                t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../try_infhelix.py ')
                    fpbs.write(' -infhelix_maxtheta %f ' % maxtheta)
                    fpbs.write(' -infhelix_nnode %f ' % nphi)
                    fpbs.write(' -infhelix_ntheta %f ' % nth)
                    fpbs.write(' -ksp_max_it 1000')
                    fpbs.write(' > %s.txt' % job_name)
                    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
    
    frun.write('\n')


In [61]:
# case 3, comparation with Liu Bin
#  helical swimming in stokes flow using a novel boundary-element method
#    Fig 5
a_tao = [(0.013, 'a'), (0.026, 'b')]
nth = 500
ch = 40
infhelix_ntheta = ch * 500
rh1 = 1
job_dir = 'compare_Liu'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

for ta, idx in a_tao:
    job_dir2 = os.path.join(job_dir, 'bFig_5_%s' % idx)
    if not os.path.exists(job_dir2):
        os.makedirs(job_dir2)
        
    t_name = os.path.join(job_dir2, 'run.sh')
    with open(t_name, 'w') as frun:
        # helix angle, tan(theta)=(2*pi*R)/ph
        for theta in np.linspace(0, np.pi / 2, 30)[1:]:
            rh2 = ta * 2 * np.pi * rh1 / np.sin(theta)
            ph = 2 * np.pi * rh1 / np.tan(theta)
            job_name = 'theta_%4.2f' % theta
            t_name = os.path.join(job_dir2, '%s.pbs' % job_name)
            with open(t_name, 'w') as fpbs:
                write_pbs_head(fpbs, job_name)
                fpbs.write('mpirun -n 24 python ')
                fpbs.write(' ../../try_infhelix.py ')
                fpbs.write(' -nth %d ' % nth)
                fpbs.write(' -ch %f ' % ch)
                fpbs.write(' -ph %f ' % ph)
                fpbs.write(' -rh1 %f ' % rh1)
                fpbs.write(' -rh2 %f ' % rh2)
                fpbs.write(' -infhelix_ntheta %d ' % infhelix_ntheta)
                fpbs.write(' -ksp_max_it 1000')
                fpbs.write(' > %s.txt' % job_name)
                # write to .sh file
                frun.write('qsub %s.pbs\n' % job_name)
        frun.write('\n')




In [86]:
# case 4, comparation with Liu Bin
#  Propulsion by a helical flagellum in a capillary tube
#    Fig 2a
a_tao = 0.013
nth = 200
ch = 100
eh = -1
infhelix_ntheta = ch * 300
rh1 = 1
n_helix = 1
theta_list = np.arange(0.032, 0.5, 0.032)
zmfct_list = np.linspace(0.1, 1, 30, endpoint=False)
dbg_mode = False

job_dir = 'compare_Liu'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
job_dir2 = os.path.join(job_dir, 'cFig_2_a')
if not os.path.exists(job_dir2):
    os.makedirs(job_dir2)

t_name = os.path.join(job_dir2, 'run.sh')
with open(t_name, 'w') as frun:
    # helix angle, tan(theta)=(2*pi*R)/ph
    for theta in theta_list:
        job_name = 'theta_%4.2f' % theta
        t_name = os.path.join(job_dir2, '%s.pbs' % job_name)
        rh2 = a_tao * 2 * np.pi * rh1 / np.sin(theta * np.pi)
        ph = 2 * np.pi * rh1 / np.tan(theta * np.pi)
        with open(t_name, 'w') as fpbs:
            write_pbs_head(fpbs, job_name)
            # infHelix without tube
            fpbs.write('mpirun -n 24 python ')
            if dbg_mode:
                fpbs.write(' ../try_infhelix.py ')
                fpbs.write(' -plot_geo 1 ')
            else:
                fpbs.write(' ../../try_infhelix.py ')
            fpbs.write(' -nth %d ' % nth)
            fpbs.write(' -ch %f ' % ch)
            fpbs.write(' -ph %f ' % ph)
            fpbs.write(' -eh %f ' % eh)
            fpbs.write(' -rh1 %f ' % rh1)
            fpbs.write(' -rh2 %f ' % rh2)
            fpbs.write(' -n_helix %d ' % n_helix)
            fpbs.write(' -infhelix_ntheta %d ' % infhelix_ntheta)
            fpbs.write(' -ksp_max_it 1000')
            fpbs.write(' > %s_%s.txt\n\n' % (job_name, 'inf'))
            # infHelix in tube
            for zoom_factor in zmfct_list:
                fpbs.write('mpirun -n 24 python ')
                if dbg_mode:
                    fpbs.write(' ../helixInPipe.py ')
                    fpbs.write(' -plot_geo 1 ')
                else:
                    fpbs.write(' ../../helixInPipe.py ')
                fpbs.write(' -nth %d ' % nth)
                fpbs.write(' -ch %f ' % ch)
                fpbs.write(' -ph %f ' % ph)
                fpbs.write(' -eh %f ' % eh)
                fpbs.write(' -rh1 %f ' % rh1)
                fpbs.write(' -rh2 %f ' % rh2)
                fpbs.write(' -n_helix %d ' % n_helix)
                fpbs.write(' -infhelix_ntheta %d ' % infhelix_ntheta)
                fpbs.write(' -ksp_max_it 1000')
                fpbs.write(' -zoom_factor %f' % zoom_factor)
                fpbs.write(' > %s_%4.2f.txt\n\n' % (job_name, zoom_factor))
            # write to .sh file
            frun.write('qsub %s.pbs\n' % job_name)
    frun.write('\n')




In [94]:
# case 4, comparation with Liu Bin, each case generate a qsub file separately. 
#  Propulsion by a helical flagellum in a capillary tube
#    Fig 2a
a_tao = 0.013
nth = 200
ch = 100
eh = -1
infhelix_ntheta = ch * 300
rh1 = 1
n_helix = 1
theta_list = np.arange(0.032, 0.5, 0.032)
zmfct_list = np.linspace(0.1, 1, 30, endpoint=False)
dbg_mode = False

job_dir = 'compare_Liu'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)
job_dir2 = os.path.join(job_dir, 'dFig_2_a')
if not os.path.exists(job_dir2):
    os.makedirs(job_dir2)

t_name = os.path.join(job_dir2, 'run.sh')
with open(t_name, 'w') as frun:
    frun.write('t_dir=$PWD\n')

    # helix angle, tan(theta)=(2*pi*R)/ph
    for theta in theta_list:
        job_dir3 = os.path.join(job_dir2, 'theta_%4.2f' % theta)
        if not os.path.exists(job_dir3):
            os.makedirs(job_dir3)
        rh2 = a_tao * 2 * np.pi * rh1 / np.sin(theta * np.pi)
        ph = 2 * np.pi * rh1 / np.tan(theta * np.pi)
        
        t_name = os.path.join(job_dir3, 'run.sh')
        with open(t_name, 'w') as frun2:
            # infHelix without tube
            job_name = 'theta_%4.2f_inf' % (theta)
            t_name = os.path.join(job_dir3, '%s.pbs' % job_name)
            with open(t_name, 'w') as fpbs:
                write_pbs_head(fpbs, job_name)
                fpbs.write('mpirun -n 24 python ')
                if dbg_mode:
                    fpbs.write(' ../try_infhelix.py ')
                    fpbs.write(' -plot_geo 1 ')
                else:
                    fpbs.write(' ../../../try_infhelix.py ')
                fpbs.write(' -nth %d ' % nth)
                fpbs.write(' -ch %f ' % ch)
                fpbs.write(' -ph %f ' % ph)
                fpbs.write(' -eh %f ' % eh)
                fpbs.write(' -rh1 %f ' % rh1)
                fpbs.write(' -rh2 %f ' % rh2)
                fpbs.write(' -n_helix %d ' % n_helix)
                fpbs.write(' -infhelix_ntheta %d ' % infhelix_ntheta)
                fpbs.write(' -ksp_max_it 1000')
                fpbs.write(' > %s_%s.txt\n\n' % (job_name, 'inf'))
                # write to .sh file
                frun2.write('qsub %s.pbs\n' % job_name)

            # infHelix in tube
            for zoom_factor in zmfct_list:
                job_name = 'theta_%4.2f_%4.2f' % (theta, zoom_factor)
                t_name = os.path.join(job_dir3, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    if dbg_mode:
                        fpbs.write(' ../helixInPipe.py ')
                        fpbs.write(' -plot_geo 1 ')
                    else:
                        fpbs.write(' ../../../helixInPipe.py ')
                    fpbs.write(' -nth %d ' % nth)
                    fpbs.write(' -ch %f ' % ch)
                    fpbs.write(' -ph %f ' % ph)
                    fpbs.write(' -eh %f ' % eh)
                    fpbs.write(' -rh1 %f ' % rh1)
                    fpbs.write(' -rh2 %f ' % rh2)
                    fpbs.write(' -n_helix %d ' % n_helix)
                    fpbs.write(' -infhelix_ntheta %d ' % infhelix_ntheta)
                    fpbs.write(' -ksp_max_it 1000')
                    fpbs.write(' -zoom_factor %f' % zoom_factor)
                    fpbs.write(' > %s_%4.2f.txt\n\n' % (job_name, zoom_factor))
                    # write to .sh file
                    frun2.write('qsub %s.pbs\n' % job_name)
        frun.write('cd $t_dir/theta_%4.2f\n' % theta)
        frun.write('bash run.sh\n')
    frun.write('cd $t_dir\n')
    frun.write('\n')


